In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from statsforecast.models import AutoETS, AutoMFLES
from statsforecast.core import StatsForecast
from utilsforecast.evaluation import evaluate
from typing import List

sales = pd.read_csv("Phase 0 - Sales.csv", na_values=np.nan)
#sales = pd.read_csv("filtered_second_file.csv", na_values=np.nan)
df = sales.set_index(["Client", "Warehouse","Product"]).stack().reset_index()
df["unique_id"] = df["Client"].astype(str) + "/" + df["Warehouse"].astype(str) + "/" + df["Product"].astype(str)
df = df.drop(columns =  ["Client", "Warehouse", "Product"])
df.columns = ["ds", "y", "unique_id"]
df["ds"] = pd.to_datetime(df["ds"])

print(df)

                ds    y     unique_id
0       2020-07-06  7.0       0/1/367
1       2020-07-13  7.0       0/1/367
2       2020-07-20  7.0       0/1/367
3       2020-07-27  7.0       0/1/367
4       2020-08-03  7.0       0/1/367
...            ...  ...           ...
2559005 2023-09-04  4.0  46/318/14294
2559006 2023-09-11  3.0  46/318/14294
2559007 2023-09-18  5.0  46/318/14294
2559008 2023-09-25  0.0  46/318/14294
2559009 2023-10-02  0.0  46/318/14294

[2559010 rows x 3 columns]


In [12]:

# Model inputs
horizon = 13
freq = "W-MON"

seasonal_period = 52

config = {
    'seasonality_weights': [True, False],  # Allows for shifting seasonalities over time
    'smoother': [True, False],  # Exponential smoothing or simple moving average
    'ma': [seasonal_period, int(seasonal_period / 2), None],  # Moving average order
    'seasonal_period': [None, seasonal_period],  # Seasonal period
}

model = AutoMFLES(season_length=52,
                  test_size=26,
                  n_windows=2,
                  metric='mse',
                  
config=config #We can pass a custom config here
                  )

sf = StatsForecast(
                  models=[model],
                  freq=freq,
                  n_jobs=-1,
                  verbose=True
                          )

# Fit the model, takes about 15 mins to tune and forecast on a basic google colab
sf.fit(df = df)

# Generate test predictions
yhat_test = sf.predict(h=horizon)
yhat_test = yhat_test.reset_index()

# Enforce non-negativity
yhat_test[str(sf.models[0])] = yhat_test[str(sf.models[0])].clip(0)

# We use the name of the model
name = "submitsept27_01.csv"

# Apply some data wrangling to ensure everything is in the expected format of the competition
yhat_test[["Client", "Warehouse", "Product"]] = yhat_test["unique_id"].str.split("/", expand=True)
yhat_test[["Client", "Warehouse", "Product"]] = yhat_test[["Client", "Warehouse", "Product"]].astype(np.int64)
yhat_test = yhat_test.sort_values(by=["Client", "Warehouse", "Product"]).reset_index(drop=True)
yhat_test = yhat_test.drop(columns = "unique_id")
yhat_test = yhat_test.set_index(["Client", "Warehouse", "Product", "ds"])
yhat_test = yhat_test.unstack(3)
yhat_test.columns = yhat_test.columns.get_level_values(1)

yhat_test.to_csv(name)
yhat_test

ds                        2023-10-09  2023-10-16  2023-10-23  2023-10-30  \
Client Warehouse Product                                                   
0      1         367        3.459377    3.439253    3.419130    3.399006   
                 639       10.017445   10.019529   10.021613   10.023696   
                 655       28.841160   28.764721   28.688282   28.611843   
                 1149       0.774829    0.752534    0.730240    0.707946   
                 1485       7.558570    7.558570    7.558570    7.558570   
...                              ...         ...         ...         ...   
46     318       13485     67.078634   69.694763   69.128018   64.501081   
                 13582     29.831642   29.831642   29.831642   29.831642   
                 13691      3.411435    4.411066    5.266404    5.971739   
                 13946      3.990519    3.345492    2.426895    2.091012   
                 14294      0.000000    0.097159    0.080812    0.000000   

ds                        2023-11-06  2023-11-13  2023-11-20  2023-11-27  \
Client Warehouse Product                                                   
0      1         367        3.378882    3.358758    3.338635    3.318511   
                 639       10.025780   10.027864   10.029947   10.032031   
                 655       28.535404   28.458965   28.382526   28.306087   
                 1149       0.685651    0.663357    0.641063    0.618768   
                 1485       7.558570    7.558570    7.558570    7.558570   
...                              ...         ...         ...         ...   
46     318       13485     59.434819   58.401192   61.567233   64.355429   
                 13582     29.831642   29.831642   29.831642   29.831642   
                 13691      6.795024    7.768564    8.382285    7.931121   
                 13946      2.624112    3.279136    3.217179    2.577309   
                 14294      0.000000    0.000000    0.086796    0.014526   

ds                        2023-12-04  2023-12-11  2023-12-18  2023-12-25  \
Client Warehouse Product                                                   
0      1         367        3.298387    3.278264    3.258140    3.238016   
                 639       10.034115   10.036199   10.038282   10.040366   
                 655       28.229648   28.153209   28.076771   28.000332   
                 1149       0.596474    0.574179    0.551885    0.529591   
                 1485       7.558570    7.558570    7.558570    7.558570   
...                              ...         ...         ...         ...   
46     318       13485     62.412613   56.282276   50.562670   48.909967   
                 13582     29.831642   29.831642   29.831642   29.831642   
                 13691      6.215222    3.866569    1.943784    1.165724   
                 13946      2.242641    2.641856    3.201615    3.155370   
                 14294      0.000000    0.000000    0.388263    0.996750   

ds                        2024-01-01  
Client Warehouse Product              
0      1         367        3.217893  
                 639       10.042450  
                 655       27.923893  
                 1149       0.507296  
                 1485       7.558570  
...                              ...  
46     318       13485     50.804992  
                 13582     29.831642  
                 13691      1.471681  
                 13946      2.584870  
                 14294      1.210625  

[15053 rows x 13 columns]